In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
mnist = input_data.read_data_sets('MNIST_data/',one_hot=True)

W1009 12:19:46.067090 18392 deprecation.py:323] From <ipython-input-2-d08c12c637be>:1: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W1009 12:19:46.072076 18392 deprecation.py:323] From C:\Users\sharma ji\AppData\Roaming\Python\Python37\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W1009 12:19:46.075067 18392 deprecation.py:323] From C:\Users\sharma ji\AppData\Roaming\Python\Python37\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:262: extract_images (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a futur

Extracting MNIST_data/train-images-idx3-ubyte.gz


W1009 12:19:46.434141 18392 deprecation.py:323] From C:\Users\sharma ji\AppData\Roaming\Python\Python37\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.
W1009 12:19:46.437094 18392 deprecation.py:323] From C:\Users\sharma ji\AppData\Roaming\Python\Python37\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:110: dense_to_one_hot (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.one_hot on tensors.
W1009 12:19:46.505887 18392 deprecation.py:323] From C:\Users\sharma ji\AppData\Roaming\Python\Python37\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.

Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


## Creating necessary functions

In [3]:
# input will be in format like [batch,height,width,color_channels]
# weights will be in format like [filter height,filter width,channels_in,channels_out]

def init_weights(shape):
    w = tf.truncated_normal(shape=shape,stddev=0.1)
    return tf.Variable(w)

def init_biases(shape):
    b = tf.constant(value=0.1,shape=shape)
    return tf.Variable(b)

def conv2d(input_image,W):
    return tf.nn.conv2d(input_image,W,strides=[1,1,1,1],padding='SAME')

def max_pool_2by2(input_layer):
    return tf.nn.max_pool(value=input_layer,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

def convolution_layer(input_image,filter_shape):
    weights = init_weights(filter_shape)
    biases = init_biases([filter_shape[3]])
    return tf.nn.relu(conv2d(input_image,weights) + biases)

def normal_layer(inputs,no_of_nodes):
    shape = int(inputs.get_shape()[1])
    weights = init_weights([shape,no_of_nodes])
    biases = init_biases([no_of_nodes])
    return tf.matmul(inputs,weights) + biases

## Creating placeholder to feed data in the nn

In [4]:
x       = tf.placeholder(tf.float32,shape=[None,784])
y_true  = tf.placeholder(tf.float32,shape=[None,10])
input_image = tf.reshape(x,[-1,28,28,1]) 

## Creating layers 

In [5]:
convo_1 = convolution_layer(input_image,[5,5,1,32])
convo_1_pool = max_pool_2by2(convo_1)

convo_2 = convolution_layer(convo_1_pool,[5,5,32,64])
convo_2_pool = max_pool_2by2(convo_2)

convo_2_flattened = tf.reshape(convo_2_pool,[-1,7*7*64])
full_layer_one = tf.nn.relu(normal_layer(convo_2_flattened,1024))

# Doing dropout
hold_prob = tf.placeholder(tf.float32)
full_layer_one_with_dropping = tf.nn.dropout(full_layer_one,hold_prob)


W1009 12:19:46.899137 18392 deprecation.py:506] From <ipython-input-5-924dba997d2a>:12: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [6]:
y_pred = normal_layer(full_layer_one_with_dropping,10)

## Loss Function

In [7]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true,logits=y_pred))

W1009 12:19:46.940994 18392 deprecation.py:323] From <ipython-input-7-62d7729af131>:1: softmax_cross_entropy_with_logits (from tensorflow.python.ops.nn_ops) is deprecated and will be removed in a future version.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



## Optimizer

In [8]:
optimize = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)

In [9]:
initiliaze_variables = tf.global_variables_initializer()

In [10]:
sess = tf.InteractiveSession()
sess.run(initiliaze_variables)

In [11]:
batch_size = 50
for steps in range(5000):
    input_batch,target_batch= mnist.train.next_batch(batch_size,shuffle=True)
    sess.run([optimize],feed_dict={x:input_batch,y_true:target_batch,hold_prob:0.5})
    matches = tf.equal(tf.argmax(y_pred,1),tf.argmax(y_true,1))
    acc = tf.reduce_mean(tf.cast(matches,tf.float32))
    
    if steps%100==0:
        print('On step: {}'.format(steps)+' accuracy is: ')
        print(sess.run(acc,feed_dict={x:mnist.test.images,y_true:mnist.test.labels,hold_prob:1.0}))

On step: 0 accuracy is: 
0.1556
On step: 100 accuracy is: 
0.9461
On step: 200 accuracy is: 
0.9629
On step: 300 accuracy is: 
0.97
On step: 400 accuracy is: 
0.9764
On step: 500 accuracy is: 
0.9777
On step: 600 accuracy is: 
0.9775
On step: 700 accuracy is: 
0.9823
On step: 800 accuracy is: 
0.9818
On step: 900 accuracy is: 
0.9825
On step: 1000 accuracy is: 
0.9833
On step: 1100 accuracy is: 
0.9861
On step: 1200 accuracy is: 
0.9817
On step: 1300 accuracy is: 
0.9849
On step: 1400 accuracy is: 
0.98
On step: 1500 accuracy is: 
0.986
On step: 1600 accuracy is: 
0.9843
On step: 1700 accuracy is: 
0.9869
On step: 1800 accuracy is: 
0.9886
On step: 1900 accuracy is: 
0.9873
On step: 2000 accuracy is: 
0.9882
On step: 2100 accuracy is: 
0.9865
On step: 2200 accuracy is: 
0.985
On step: 2300 accuracy is: 
0.9905
On step: 2400 accuracy is: 
0.99
On step: 2500 accuracy is: 
0.9899
On step: 2600 accuracy is: 
0.9893
On step: 2700 accuracy is: 
0.9875
On step: 2800 accuracy is: 
0.9887
On st